In [2]:
import math

## Define states and mapping

In [4]:
states = ['E', '5', 'I']
state_to_idx = {s: i for i, s in enumerate(states)}

## Transition Probabilities in Hidden Markov Models (HMM)

### 🔹 Definition
Transition probabilities describe the likelihood of moving from one hidden state to another in an HMM.

They are written as:

$$
P(s_t = j \mid s_{t-1} = i)
$$

Where:

- \( s_t \): hidden state at time \( t \)
- \( s_{t-1} \): hidden state at time \( t-1 \)

---

### 🔹 Key Points

- They model how the system changes over time by capturing dependencies between consecutive hidden states.
- For any given current state \( i \), the total probability of transitioning to all possible next states \( j \) must sum to 1:

$$
\sum_{j} P(s_t = j \mid s_{t-1} = i) = 1
$$

---

### 🔹 Example: 5' Splice Site Detection in DNA

Suppose we have the following HMM states:
- **E**: Exon
- **5**: 5' splice site
- **I**: Intron

Then the transition probabilities could be:

- From Exon to Exon:

  $$
  P(E \rightarrow E) = 0.9
  $$

- From Exon to 5' splice site:

  $$
  P(E \rightarrow 5) = 0.1
  $$

- From 5' splice site to Intron:

  $$
  P(5 \rightarrow I) = 1.0
  $$


## Transition matrix T[k][j] = P(from k to j)

In [7]:
T = [
    [0.9, 0.1, 0],  # E to E, 5, I
    [0, 0, 1],      # 5 to E, 5, I
    [0, 0, 0.9]     # I to E, 5, I
]
print(T)

[[0.9, 0.1, 0], [0, 0, 1], [0, 0, 0.9]]


## probabilty to end state(only possible I_end)

Intial state is always E so init=[1,0,0]

In [10]:
P_to_end = [0, 0, 0.1]  
init = [1, 0, 0] 

## Emission Probabilities in Hidden Markov Models (HMM)

### 🔹 Definition
Emission probabilities represent the likelihood of observing a specific symbol (e.g., a nucleotide like A, C, G, or T) given the current hidden state.

They are written as:

$$
P(o_t = k \mid s_t = i)
$$

Where:

- \( o_t \): observed symbol at time \( t \)
- \( s_t \): hidden state at time \( t \)

---

### 🔹 Key Points

- Emission probabilities link the **hidden states** to the **observable data**.
- For any given state \( i \), the sum of emission probabilities over all possible symbols \( k \) must be 1:

$$
\sum_k P(o_t = k \mid s_t = i) = 1
$$

---

## Example: 5' Splice Site Detection in DNA

### Exon State (E)
Assume equal likelihood for all nucleotides:

- \( P(A \| E) = 0.25 \)
- \( P(C \| E) = 0.25 \)
- \( P(G \| E) = 0.25 \)
- \( P(T \| E) = 0.25 \)

### 5' Splice Site State (5)
Assume preference for G:

- \( P(G \| 5) = 0.95 \)
- \( P(A \| 5) = 0.05 \)
- \( P(C \| 5) = 0 \)
- \( P(T \| 5) = 0 \)

### Intron State (I)
Assume equal likelihood for all nucleotides:

- \( P(A \| I) = 0.25 \)
- \( P(C \| I) = 0.25 \)
- \( P(G \| I) = 0.25 \)
- \( P(T \| I) = 0.25 \)



## Emission prob matrix

In [13]:
E = [
    {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},  # E
    {'A': 0.05, 'C': 0, 'G': 0.95, 'T': 0},       # 5
    {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4}      # I
]
print(E)

[{'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}, {'A': 0.05, 'C': 0, 'G': 0.95, 'T': 0}, {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4}]


## Calculating log probability of a given state path for the sequence.

In [15]:
def get_log_prob_of_a_given_path(path_str, sequence):
    L = len(sequence)
    path = [state_to_idx[s] for s in path_str]
    pi1 = path[0]
    if init[pi1] == 0 or E[pi1].get(sequence[0], 0) == 0:
        return float('-inf')
    log_p = math.log(init[pi1]) + math.log(E[pi1][sequence[0]])

    for i in range(1, L):
        prev_state = path[i-1]
        curr_state = path[i]
        trans_prob = T[prev_state][curr_state]
        emit_prob = E[curr_state].get(sequence[i], 0)
        if trans_prob == 0 or emit_prob == 0:
            return float('-inf')
        log_p += math.log(trans_prob) + math.log(emit_prob)
    last_state = path[-1]
    p_to_end = P_to_end[last_state]
    if p_to_end == 0:
        return float('-inf')
    log_p += math.log(p_to_end)
    
    return log_p

# Viterbi Algorithm-Implementation

In [17]:
def viterbi(sequence):
    """Find the most likely state path using the Viterbi algorithm."""
    L = len(sequence)
    N = len(states)
    V = [[float('-inf') for _ in range(N)] for _ in range(L)]
    bp = [[0 for _ in range(N)] for _ in range(L)]
    s1 = sequence[0]
    for k in range(N):
        if init[k] > 0 and E[k].get(s1, 0) > 0:
            V[0][k] = math.log(init[k]) + math.log(E[k][s1])

    for i in range(1, L):
        si = sequence[i]
        for j in range(N):
            if E[j].get(si, 0) > 0:
                max_log_p = float('-inf')
                best_k = -1
                for k in range(N):
                    if T[k][j] > 0:
                        log_p = V[i-1][k] + math.log(T[k][j])
                        if log_p > max_log_p:
                            max_log_p = log_p
                            best_k = k
                if best_k != -1:
                    V[i][j] = max_log_p + math.log(E[j][si])
                    bp[i][j] = best_k
    
    max_log_p = float('-inf')
    best_last_k = -1
    for k in range(N):
        if P_to_end[k] > 0 and V[L-1][k] != float('-inf'):
            log_p = V[L-1][k] + math.log(P_to_end[k])
            if log_p > max_log_p:
                max_log_p = log_p
                best_last_k = k
    
    if best_last_k == -1:
        return None, float('-inf')
     # Backtracking to reconstruct the path
    path = [best_last_k]
    for i in range(L-1, 0, -1):
        prev_k = bp[i][path[0]]
        path.insert(0, prev_k)
    
    path_str = ''.join(states[k] for k in path)
    return path_str, max_log_p

## Testing with Example Given

In [19]:
sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
specific_path ="EEEEEEEEEEEEEEEEEE5IIIIIII"
log_p_specific = get_log_prob_of_a_given_path(specific_path, sequence)
print(f"Log probability of path '{specific_path}': {log_p_specific:.2f}")

Log probability of path 'EEEEEEEEEEEEEEEEEE5IIIIIII': -41.22


## Most Likely Path

In [21]:
path, log_p = viterbi(sequence)
print(f"Most likely path: {path}")
print(f"Log probability: {log_p:.2f}")

Most likely path: EEEEEEEEEEEEEEEEEE5IIIIIII
Log probability: -41.22
